In [2]:
import pandas as pd
import numpy as np

C:\Users\ghtjq\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
df = pd.read_csv('data/시군구코드.csv', encoding='utf-8')
df

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM
0,20231231,11010,종로구
1,20231231,11020,중구
2,20231231,11030,용산구
3,20231231,11040,성동구
4,20231231,11050,광진구
...,...,...,...
245,20231231,38580,함양군
246,20231231,38590,거창군
247,20231231,38600,합천군
248,20231231,39010,제주시


In [4]:
sido = pd.read_csv('data/시도코드.csv', encoding = 'cp949', header = None, names = ['시도', '코드'])
sido

,시도,코드
0,서울특별시,11
1,부산광역시,21
2,대구광역시,22
3,인천광역시,23
4,광주광역시,24
5,대전광역시,25
6,울산광역시,26
7,세종특별자치시,29
8,경기도,31
9,강원도,32


In [18]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME") 
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [26]:
from sqlalchemy import create_engine, text
import pymysql
import pandas as pd
db_connection_str = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()

bank_addr = pd.read_sql_query(text("SELECT * FROM bank_address"), conn)

In [90]:
bank_addr.to_csv('data/bank_address.csv', index=False)

In [91]:
bank_addr = pd.read_csv("data/bank_address.csv")
bank_addr

,은행,지점,주소
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895
...,...,...,...
5529,한국씨티은행,울산,울산광역시 남구 달동 번영로
5530,한국씨티은행,제주,제주특별자치도 제주시 노형동 도령로
5531,한국씨티은행,중앙기업업무센터,서울특별시 종로구 신문로2가 새문안로
5532,한국씨티은행,청주,충청북도 청주시 서원구 1순환로


In [92]:
bank_addr['시도'] = bank_addr['주소'].apply(lambda x: '강원도' if x.split()[0] == '강원특별자치도' 
                                                      else '전라북도' if x.split()[0] == '전북특별자치도'
                                                      else x.split()[0])
bank_addr.head()

,은행,지점,주소,시도
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥,경상남도
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로,서울특별시
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36,경상남도
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421,서울특별시
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895,경상남도


In [93]:
bank_addr = pd.merge(bank_addr, sido)
bank_addr.head()

,은행,지점,주소,시도,코드
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥,경상남도,38
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로,서울특별시,11
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36,경상남도,38
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421,서울특별시,11
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895,경상남도,38


In [94]:
bank_addr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5534 entries, 0 to 5533
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   은행      5534 non-null   object
 1   지점      5534 non-null   object
 2   주소      5534 non-null   object
 3   시도      5534 non-null   object
 4   코드      5534 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 216.3+ KB


In [95]:
bank_addr.시도.unique()

array(['경상남도', '서울특별시', '울산광역시', '부산광역시', '경기도', '경상북도', '대구광역시', '대전광역시',
       '인천광역시', '강원도', '광주광역시', '전라남도', '전라북도', '충청남도', '제주특별자치도',
       '세종특별자치시', '충청북도'], dtype=object)

In [96]:
sido.시도.unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [97]:
import re

def extract_sigungu_regex(address):
    # 세종특별자치시 처리
    if address.startswith('세종특별자치시'):
        return '세종시'
    
    # 시도명 패턴
    sido_pattern = r'(서울특별시|부산광역시|대구광역시|인천광역시|광주광역시|대전광역시|울산광역시|세종특별자치시|경기도|강원도|강원특별자치도|충청북도|충청남도|전라북도|전북특별자치도|전라남도|경상북도|경상남도|제주특별자치도)'
    
    # 모든 시도명 위치 찾기
    matches = list(re.finditer(sido_pattern, address))
    
    if len(matches) >= 2:  # 중복된 시도명이 있는 경우
        # 두 번째 시도명 이후부터 처리
        second_sido_end = matches[1].end()
        remaining = address[second_sido_end:].strip().split()
        
        # 시군구 추출
        if len(remaining) >= 1:
            sigungu = remaining[0]
            if len(remaining) >= 2 and remaining[1].endswith('구'):
                sigungu += ' ' + remaining[1]
            return sigungu
    
    # 일반적인 경우
    parts = address.split()
    sigungu = parts[1] if len(parts) > 1 else ''
    if len(parts) > 2 and parts[2].endswith('구'):
        sigungu += ' ' + parts[2]
    return sigungu

bank_addr['시군구'] = bank_addr['주소'].apply(extract_sigungu_regex)

In [162]:
bank_addr['시군구'] = bank_addr['시군구'].replace({
    '안산시': '안산시 단원구',
    '청주시': '청주시 흥덕구', 
    '수원시': '수원시 영통구',
    '창원시': '창원시 성산구',
    '전주시': '전주시 완산구',
    '포항시': '포항시 북구',
    '부천시 원미구' : '부천시',
    '부천시 소사구' : '부천시',
    '부천시 오정구' : '부천시',
    '고양시 일산구': '고양시 일산동구',
    '서구 서구': '서구'
})

In [163]:
bank_addr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5534 entries, 0 to 5533
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   은행      5534 non-null   object
 1   지점      5534 non-null   object
 2   주소      5534 non-null   object
 3   시도      5534 non-null   object
 4   코드      5534 non-null   int64 
 5   시군구     5534 non-null   object
dtypes: int64(1), object(5)
memory usage: 259.5+ KB


In [164]:
bank_addr[bank_addr['시군구'] == '서구 서구']
# bank_addr[bank_addr['주소'].str.contains('대전광역시 서구')]

,은행,지점,주소,시도,코드,시군구


In [42]:
# bank_addr['시군구'] = bank_addr['주소'].apply(
#     lambda x: '세종시' if x.split()[0] == '세종특별자치시'
#     else x.split()[1] + (' ' + x.split()[2] if x.split()[2].endswith('구') else '')
# )
# bank_addr

,은행,지점,주소,시도,코드,시군구
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥,경상남도,38,진주시
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로,서울특별시,11,금천구
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36,경상남도,38,창원시 성산구
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421,서울특별시,11,강남구
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895,경상남도,38,거제시
...,...,...,...,...,...,...
5529,한국씨티은행,울산,울산광역시 남구 달동 번영로,울산광역시,26,남구
5530,한국씨티은행,제주,제주특별자치도 제주시 노형동 도령로,제주특별자치도,39,제주시
5531,한국씨티은행,중앙기업업무센터,서울특별시 종로구 신문로2가 새문안로,서울특별시,11,종로구
5532,한국씨티은행,청주,충청북도 청주시 서원구 1순환로,충청북도,33,청주시 서원구


In [146]:
# [STEP 1] df에 '시도코드' 열 추가하기
# 'SIGUNGU_CD'의 앞 두 자리를 '시도코드'로 사용합니다.
df['시도코드'] = df['SIGUNGU_CD'].astype(str).str[:2].astype(int)
df

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,시도코드
0,20231231,11010,종로구,11
1,20231231,11020,중구,11
2,20231231,11030,용산구,11
3,20231231,11040,성동구,11
4,20231231,11050,광진구,11
...,...,...,...,...
245,20231231,38580,함양군,38
246,20231231,38590,거창군,38
247,20231231,38600,합천군,38
248,20231231,39010,제주시,39


In [165]:
# [STEP 2] 복합 키로 Merge 수행
# '시도코드'와 '시군구' 이름을 동시에 사용하여 merge합니다.
# how='left' 옵션으로 bank_addr의 모든 데이터를 보존합니다.
final_df = pd.merge(bank_addr,
                    df[['SIGUNGU_CD', 'SIGUNGU_NM', '시도코드']], # df에서는 필요한 열만 선택
                    left_on=['코드', '시군구'],
                    right_on=['시도코드', 'SIGUNGU_NM'],
                    how='left')

In [166]:
final_df[final_df['시도코드'].isnull()]

,은행,지점,주소,시도,코드,시군구,SIGUNGU_CD,SIGUNGU_NM,시도코드
3094,Sh수협은행,학익동,인천광역시 남구 학익동 소성로 197,인천광역시,23,남구,NaN,NaN,NaN


In [167]:
final_df[final_df['시도코드'].isnull()]['시군구'].unique()

array(['남구'], dtype=object)

In [168]:
final_df.info() # 시군구 코드 결측치 1개로 줄임

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5534 entries, 0 to 5533
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   은행          5534 non-null   object 
 1   지점          5534 non-null   object 
 2   주소          5534 non-null   object 
 3   시도          5534 non-null   object 
 4   코드          5534 non-null   int64  
 5   시군구         5534 non-null   object 
 6   SIGUNGU_CD  5533 non-null   float64
 7   SIGUNGU_NM  5533 non-null   object 
 8   시도코드        5533 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 389.2+ KB


In [169]:
final_df.dropna(axis=0, inplace=True)

In [170]:
# [STEP 3] 결과 확인
# 불필요하게 중복되는 SIGUNGU_NM 열은 삭제하여 깔끔하게 만듭니다.
final_df = final_df.drop(columns=['SIGUNGU_NM', '시도코드'])

print("### 최종 Merge 결과 ###")
print(final_df)

### 최종 Merge 결과 ###
           은행          지점                           주소       시도  코드      시군구  \
0     BNK경남은행          LH  경상남도 진주시 충무공동 충의로 19 LH진주사옥     경상남도  38      진주시   
1     BNK경남은행       가산디지털       서울특별시 금천구 가산동 가산디지털 1로    서울특별시  11      금천구   
2     BNK경남은행     가음정금융센터          경상남도 창원시 성산구 대정로 36     경상남도  38  창원시 성산구   
3     BNK경남은행          강남       서울특별시 강남구 대치동 영동대로 421    서울특별시  11      강남구   
4     BNK경남은행        거제고현      경상남도 거제시 고현동 거제중앙로 1895     경상남도  38      거제시   
...       ...         ...                          ...      ...  ..      ...   
5529   한국씨티은행          울산              울산광역시 남구 달동 번영로    울산광역시  26       남구   
5530   한국씨티은행          제주          제주특별자치도 제주시 노형동 도령로  제주특별자치도  39      제주시   
5531   한국씨티은행    중앙기업업무센터         서울특별시 종로구 신문로2가 새문안로    서울특별시  11      종로구   
5532   한국씨티은행          청주            충청북도 청주시 서원구 1순환로     충청북도  33  청주시 서원구   
5533   한국씨티은행  커머셜기업금융1센터         서울특별시 종로구 신문로2가 새문안로    서울특별시  11      종로구   

      SIGUNGU_CD  


In [171]:
final_df

,은행,지점,주소,시도,코드,시군구,SIGUNGU_CD
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥,경상남도,38,진주시,38030.0
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로,서울특별시,11,금천구,11180.0
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36,경상남도,38,창원시 성산구,38112.0
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421,서울특별시,11,강남구,11230.0
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895,경상남도,38,거제시,38090.0
...,...,...,...,...,...,...,...
5529,한국씨티은행,울산,울산광역시 남구 달동 번영로,울산광역시,26,남구,26020.0
5530,한국씨티은행,제주,제주특별자치도 제주시 노형동 도령로,제주특별자치도,39,제주시,39010.0
5531,한국씨티은행,중앙기업업무센터,서울특별시 종로구 신문로2가 새문안로,서울특별시,11,종로구,11010.0
5532,한국씨티은행,청주,충청북도 청주시 서원구 1순환로,충청북도,33,청주시 서원구,33042.0


In [172]:
final_df.to_csv('data/은행주소_시군구코드추가.csv', index=False)

In [173]:
df = pd.read_csv('data/은행주소_시군구코드추가.csv')
df

,은행,지점,주소,시도,코드,시군구,SIGUNGU_CD
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥,경상남도,38,진주시,38030.0
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로,서울특별시,11,금천구,11180.0
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36,경상남도,38,창원시 성산구,38112.0
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421,서울특별시,11,강남구,11230.0
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895,경상남도,38,거제시,38090.0
...,...,...,...,...,...,...,...
5528,한국씨티은행,울산,울산광역시 남구 달동 번영로,울산광역시,26,남구,26020.0
5529,한국씨티은행,제주,제주특별자치도 제주시 노형동 도령로,제주특별자치도,39,제주시,39010.0
5530,한국씨티은행,중앙기업업무센터,서울특별시 종로구 신문로2가 새문안로,서울특별시,11,종로구,11010.0
5531,한국씨티은행,청주,충청북도 청주시 서원구 1순환로,충청북도,33,청주시 서원구,33042.0


In [189]:
import re

def create_standard_address_simple(address):
    """간단하고 효과적인 표준 주소 생성"""
    
    # 중복 시도명 제거
    sido_pattern = r'(서울특별시|부산광역시|대구광역시|인천광역시|광주광역시|대전광역시|울산광역시|세종특별자치시|경기도|강원도|강원특별자치도|충청북도|충청남도|전라북도|전북특별자치도|전라남도|경상북도|경상남도|제주특별자치도)'
    
    matches = list(re.finditer(sido_pattern, address))
    if len(matches) >= 2:
        address = address[matches[1].start():]
    
    # 쉼표 제거 (도로명 뒤에 자주 나타남)
    address = address.replace(',', '')
    
    # 건물명, 층수 등 제거
    # 숫자 + 건물관련단어 이후 모든 내용 제거
    clean_address = re.sub(r'\s+\d+\s+[가-힣]*(?:빌딩|타워|센터|사옥|아파트|오피스텔).*$', '', address)
    clean_address = re.sub(r'\s+[가-힣]*(?:빌딩|타워|센터|사옥|아파트|오피스텔).*$', '', clean_address)
    clean_address = re.sub(r'\s+\d+층.*$', '', clean_address)
    
    # 공백 정리
    clean_address = re.sub(r'\s+', ' ', clean_address).strip()
    
    # 기본적으로 시도 + 시군구 + 구/동 + 도로명 + 번지수까지 유지
    parts = clean_address.split()
    
    # 최대 5개 요소까지 (시도, 시군구, 구/동, 도로명, 번지수)
    if len(parts) > 5:
        # 번지수가 있는지 확인
        for i in range(len(parts)):
            if re.match(r'^\d+(-\d+)?$', parts[i]):
                result_parts = parts[:i+1]  # 번지수까지 포함
                return ' '.join(result_parts)
        # 번지수가 없으면 앞의 5개만
        return ' '.join(parts[:5])
    
    return clean_address

# 적용
df['표준주소'] = df['주소'].apply(create_standard_address_simple)

In [190]:
# 결과 확인
print("수정된 원본 주소 vs 표준 주소 비교:")
for i in range(5):
    print(f"원본: {bank_addr.iloc[i]['주소']}")
    print(f"표준: {bank_addr.iloc[i]['표준주소']}")
    print("-" * 50)

수정된 원본 주소 vs 표준 주소 비교:
원본: 경상남도 진주시 충무공동 충의로 19 LH진주사옥
표준: 경상남도 진주시 충무공동 충의로 19
--------------------------------------------------
원본: 서울특별시 금천구 가산동 가산디지털 1로
표준: 서울특별시 금천구 가산동 가산디지털 1로
--------------------------------------------------
원본: 경상남도 창원시 성산구 대정로 36
표준: 경상남도 창원시 성산구 대정로 36
--------------------------------------------------
원본: 서울특별시 강남구 대치동 영동대로 421
표준: 서울특별시 강남구 대치동 영동대로 421
--------------------------------------------------
원본: 경상남도 거제시 고현동 거제중앙로 1895
표준: 경상남도 거제시 고현동 거제중앙로 1895
--------------------------------------------------


In [191]:
df

,은행,지점,주소,시도,코드,시군구,SIGUNGU_CD,표준주소
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥,경상남도,38,진주시,38030.0,경상남도 진주시 충무공동 충의로 19
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로,서울특별시,11,금천구,11180.0,서울특별시 금천구 가산동 가산디지털 1로
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36,경상남도,38,창원시 성산구,38112.0,경상남도 창원시 성산구 대정로 36
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421,서울특별시,11,강남구,11230.0,서울특별시 강남구 대치동 영동대로 421
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895,경상남도,38,거제시,38090.0,경상남도 거제시 고현동 거제중앙로 1895
...,...,...,...,...,...,...,...,...
5528,한국씨티은행,울산,울산광역시 남구 달동 번영로,울산광역시,26,남구,26020.0,울산광역시 남구 달동 번영로
5529,한국씨티은행,제주,제주특별자치도 제주시 노형동 도령로,제주특별자치도,39,제주시,39010.0,제주특별자치도 제주시 노형동 도령로
5530,한국씨티은행,중앙기업업무센터,서울특별시 종로구 신문로2가 새문안로,서울특별시,11,종로구,11010.0,서울특별시 종로구 신문로2가 새문안로
5531,한국씨티은행,청주,충청북도 청주시 서원구 1순환로,충청북도,33,청주시 서원구,33042.0,충청북도 청주시 서원구 1순환로


In [192]:
df.to_csv('data/bank_address_cleaned.csv', index=False)

In [193]:
df[['표준주소']].rename(columns={'표준주소':'addr'}).to_csv('data/좌표 변환용.csv', index=False)